<a href="https://colab.research.google.com/github/KKAARRIIMM15/NLP/blob/main/Retrieval-Augmented%20Generation/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipedia sentence-transformers faiss-cpu

In [20]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer

pipe = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B", device_map="auto")
model = SentenceTransformer('all-MiniLM-L6-v2')

Device set to use cuda:0


In [29]:
import wikipedia
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


#topic = "Artificial intelligence"
#topic = "United States"
topic = "Deep Learning"
wiki_content = wikipedia.page(topic).content

def chunk_text(text, chunk_size=200, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

def retrieve_top_k(query, k=6):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k)
    return [chunks[i] for i in I[0]]

def rag_query(query, pipe):
    top_chunks = retrieve_top_k(query)
    #print("chuuuuuung")
    #print(top_chunks)
    context = "\n\n".join(top_chunks)
    prompt = f"""Use the context below to answer the question.

Context:
{context}

Question:
{query}

Answer:"""

    response = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)
    return response[0]['generated_text']

chunks = chunk_text(wiki_content)
#print(len(chunks[1].split(" ") ))
print("\n")
#model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

#query = "What are the main goals of Artificial Intelligence?"
query = "Where are the types of Deep Learning?"
#query = "What is the GDP of USA?"
print("\n")
answer = rag_query(query, pipe)
print("\n" + answer)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.





Use the context below to answer the question.

Context:
Deep learning is a subset of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised. Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug